In [ ]:
## Import necessary modules
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import metrics
#from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve, matthews_corrcoef
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sn
import seaborn as sns
import pandas as pd
import csv
import math
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score,KFold,cross_val_predict
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
import sklearn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks')

In [ ]:
df.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
# Initialise the Scaler
scaler = StandardScaler()
# To scale data
scaler.fit(df)

In [ ]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(999, inplace=True)

In [ ]:
# Split the dataset into features (X) and labels (y)
X = df.iloc[:, 0:167].values
y = df.iloc [:, 167].values
print(X.shape)
print(y.shape)

In [ ]:
# Split into training and test set
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
             X, y, test_size = 0.2, random_state=42)
print (len(X_train),len(X_test),len(y_train),len(y_test))

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix, roc_auc_score
from keras.models import Sequential
from keras.layers import Bidirectional, Dense, MaxPooling1D
from keras.models import Model

input_shape = (X_train.shape[1], 1)
import tensorflow as tf
import tensorflow as tf


# Define the attention mechanism as a custom layer
class AttentionLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
        # Create the Dense layer for computing attention scores during initialization
        self.attention_dense = Dense(1, activation='tanh')

    def call(self, inputs):
        # Compute attention scores
        attention_scores = self.attention_dense(inputs)
        attention_weights = tf.keras.layers.Softmax(axis=1)(attention_scores)

        # Apply attention weights to the input
        weighted_inputs = inputs * attention_weights
        return tf.reduce_sum(weighted_inputs, axis=1, keepdims=True)  # Ensure output is 2D

# Define the model with an input layer
inputs = Input(shape=(X_train.shape[1],))
x = Dense(128, activation='relu')(inputs)
x = Dropout(0.1)(x)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
# Apply the Attention Layer
x = AttentionLayer()(x)

# Now that the attention layer has output shape (None, 1), reshape it
x = tf.keras.layers.Flatten()(x)  # Flatten to ensure it is 2D

x = Dense(32, activation='relu')(x)
x = Dropout(0.1)(x)
outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs, outputs)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Initialize lists to store evaluation metrics for each fold
accuracy_list = []
f1_list = []
precision_list = []
recall_list = []
sensitivity_list = []
specificity_list = []
mcc_list = []
auc_list = []  # Initialize a list for AUC values

# Perform K-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=5)
for train_index, val_index in kfold.split(X_train):
    # Split the data into training and validation sets for the current fold
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    # Train the model on the current fold
    history=model.fit(X_train_fold, y_train_fold, epochs=40, batch_size=64, verbose=0,validation_split=0.1)

    # Evaluate the model on the validation set for the current fold
    y_val_pred = model.predict(X_val_fold)
    y_val_pred_binary = (y_val_pred > 0.5).astype(int)

    # Calculate confusion matrix for the current fold
    cm = confusion_matrix(y_val_fold, y_val_pred_binary)
    # Calculate AUC for the current fold
    auc = roc_auc_score(y_val_fold, y_val_pred)  # Use predicted probabilities for AUC calculation
    auc_list.append(auc)


    # Calculate performance evaluation metrics for the current fold
    TN = cm[0, 0]
    FP = cm[0, 1]
    FN = cm[1, 0]
    TP = cm[1, 1]

    # Accuracy
    accuracy = (TP + TN) / float(TP + TN + FP + FN)
    accuracy_list.append(accuracy)

    # F1-Score
    f1 = 2 * TP / float(2 * TP + FP + FN)
    f1_list.append(f1)

    # Precision
    precision = TP / float(TP + FP)
    precision_list.append(precision)

    # Recall
    recall = TP / float(TP + FN)
    recall_list.append(recall)

    # Sensitivity
    sensitivity = TP / float(TP + FN)
    sensitivity_list.append(sensitivity)

    # Specificity
    specificity = TN / float(TN + FP)
    specificity_list.append(specificity)

    # MCC (Matthews Correlation Coefficient)
    mcc = ((TP * TN) - (FP * FN)) / float((np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))) or 1)
    mcc_list.append(mcc)

# Calculate average performance evaluation metrics across all folds
avg_accuracy = np.mean(accuracy_list)
avg_f1 = np.mean(f1_list)
avg_precision = np.mean(precision_list)
avg_recall = np.mean(recall_list)
avg_sensitivity = np.mean(sensitivity_list)
avg_specificity = np.mean(specificity_list)
avg_mcc = np.mean(mcc_list)

# Calculate average AUC across all folds
avg_auc = np.mean(auc_list)
# Print average performance evaluation metrics
print("Accuracy =", avg_accuracy)
print("F1 Score =", avg_f1)
print("Precision =", avg_precision)
print("Recall =", avg_recall)
print("Sensitivity =", avg_sensitivity)
print("Specificity =", avg_specificity)
print("MCC =", avg_mcc)
print("AUC =", avg_auc)